<a href="https://colab.research.google.com/github/Lihini827/Diabetic-retinopathy-detection/blob/main/crossvalDensenet201(6).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://github.com/SadmanSakib93/Stratified-k-fold-cross-validation-Image-classification-keras/blob/master/stratified_K_fold_CV.ipynb

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, precision_score, confusion_matrix
from sklearn.model_selection import StratifiedKFold
from PIL import Image
import random
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,GlobalAveragePooling2D,BatchNormalization,Dropout

from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
import warnings
import os
import shutil
from PIL import ImageFile
warnings.simplefilter('error', Image.DecompressionBombWarning)
ImageFile.LOAD_TRUNCATED_IMAGES = True
from PIL import Image
Image.MAX_IMAGE_PIXELS = 1000000000
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

/usr/local/lib/python3.10/dist-packages/tensorflow/python/client/session.py:1793: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [ ]:
sourceFiles=[]
classLabels=['Mild','Moderate', 'No_DR','Proliferate_DR','Severe']
X=[]
Y=[]

img_rows, img_cols =  224, 224 # input image dimensions
train_path='/content/drive/MyDrive/auggrayscale/autobengramgrayaugtrain/'
validation_path='/content/drive/MyDrive/auggrayscale/autobengramgrayaugVal/'
test_path='/content/drive/MyDrive/auggrayscale/autobengramgrayaugtest/'

In [ ]:
datasetFolderName='/content/drive/MyDrive/auggrayscale'

In [ ]:
def transferBetweenFolders(source, dest, splitRate):
    global sourceFiles
    sourceFiles=os.listdir(source)
    if(len(sourceFiles)!=0):
        transferFileNumbers=int(len(sourceFiles)*splitRate)
        transferIndex=random.sample(range(0, len(sourceFiles)), transferFileNumbers)
        for eachIndex in transferIndex:
            shutil.move(source+str(sourceFiles[eachIndex]), dest+str(sourceFiles[eachIndex]))
    else:
        print("No file moved. Source empty!")

def transferAllClassBetweenFolders(source, dest, splitRate):
    for label in classLabels:
        transferBetweenFolders(datasetFolderName+'/'+source+'/'+label+'/',
                               datasetFolderName+'/'+dest+'/'+label+'/',
                               splitRate)

def my_metrics(y_true, y_pred):
    accuracy=accuracy_score(y_true, y_pred)
    precision=precision_score(y_true, y_pred,average='weighted')
    f1Score=f1_score(y_true, y_pred, average='weighted')
    print("Accuracy  : {}".format(accuracy))
    print("Precision : {}".format(precision))
    print("f1Score : {}".format(f1Score))
    cm=confusion_matrix(y_true, y_pred)
    print(cm)
    return accuracy, precision, f1Score

In [ ]:
transferAllClassBetweenFolders('autobengramgrayaugtest', 'autobengramgrayaugtrain', 1.0)

In [ ]:
transferAllClassBetweenFolders('autobengramgrayaugtrain', 'autobengramgrayaugtest', 0.20)

In [ ]:
def prepareNameWithLabels(folderName):
    sourceFiles=os.listdir(datasetFolderName+'/autobengramgrayaugtrain/'+folderName)
    for val in sourceFiles:
        X.append(val)
        for i in range(len(classLabels)):
          if(folderName==classLabels[i]):
              Y.append(i)

In [ ]:
for i in range(len(classLabels)):
  prepareNameWithLabels(classLabels[i])

In [ ]:
X=np.asarray(X)
Y=np.asarray(Y)

In [ ]:
seed_value= 12321
# 1. Set `PYTHONHASHSEED` environment variable at a fixed value
import os
os.environ['PYTHONHASHSEED']=str(seed_value)
# 2. Set `python` built-in pseudo-random generator at a fixed value
import random
random.seed(seed_value)
# 3. Set `numpy` pseudo-random generator at a fixed value
import numpy as np
np.random.seed(seed_value)
# 4. Set `tensorflow` pseudo-random generator at a fixed value
import tensorflow as tf
tf.random.set_seed(seed_value)

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.applications.densenet import DenseNet121
from tensorflow.keras.applications.densenet import preprocess_input
from tensorflow import keras

In [ ]:
batch_size = 128
epoch=25
IMAGE_SIZE = [224, 224]

def getModel():
  import tensorflow
  densenet =tensorflow.keras.applications.DenseNet121(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)
  x=densenet.output
  x= GlobalAveragePooling2D()(x)
  x= BatchNormalization()(x)
  x= Dropout(0.5,seed=seed_value)(x)
  x= Dense(1024,activation='relu')(x)
  x= Dense(512,activation='relu')(x)
  x= BatchNormalization()(x)
  x= Dropout(0.5, seed=seed_value)(x)

  preds=Dense(5,activation='softmax')(x)
  model=Model(inputs=densenet.input,outputs=preds)
  for layer in densenet.layers[:-8]:
    layer.trainable=False

  for layer in densenet.layers[-8:]:
    layer.trainable=True
  learning_rate = 0.0001
  optimizer1 = keras.optimizers.Adam(learning_rate=learning_rate)
  model.compile(optimizer=optimizer1 ,loss='categorical_crossentropy',metrics=['accuracy'])

  return model




model=getModel()

In [ ]:
skf = StratifiedKFold(n_splits=5, shuffle=True)
skf.get_n_splits(X, Y)
foldNum=0
for train_index, val_index in skf.split(X, Y):
    #First cut all images from validation to train (if any exists)
    transferAllClassBetweenFolders('autobengramgrayaugVal', 'autobengramgrayaugtrain', 1.0)
    foldNum+=1
    print("Results for fold",foldNum)
    X_train, X_val = X[train_index], X[val_index]
    Y_train, Y_val = Y[train_index], Y[val_index]
    # Move validation images of this fold from train folder to the validation folder
    for eachIndex in range(len(X_val)):
        classLabel=''
        for i in range(len(classLabels)):
          if(Y_val[eachIndex]==i):
              classLabel=classLabels[i]
        #Then, copy the validation images to the validation folder
        shutil.move(datasetFolderName+'/autobengramgrayaugtrain/'+classLabel+'/'+X_val[eachIndex],
                    datasetFolderName+'/autobengramgrayaugVal/'+classLabel+'/'+X_val[eachIndex])
    train_datagen = ImageDataGenerator(
                      rescale=1./255,
                      zoom_range=0.20,
                      fill_mode="nearest")
    validation_datagen = ImageDataGenerator(rescale=1./255)
    test_datagen = ImageDataGenerator(rescale=1./255)

    #Start ImageClassification Model
    train_generator = train_datagen.flow_from_directory(
        train_path,
        target_size=(img_rows, img_cols),
        batch_size=batch_size,

        class_mode='categorical',
        subset='training')

    validation_generator = validation_datagen.flow_from_directory(
        validation_path,
        target_size=(img_rows, img_cols),
        batch_size=batch_size,

        class_mode=None,  # only data, no labels
        shuffle=False)

    # fit model
    history=model.fit_generator(train_generator,
                        epochs=epoch)

    predictions = model.predict_generator(validation_generator, verbose=1)
    yPredictions = np.argmax(predictions, axis=1)
    true_classes = validation_generator.classes
    # evaluate validation performance
    print("***Performance on Validation data***")
    valAcc, valPrec, valFScore = my_metrics(true_classes, yPredictions)

Results for fold 1
Found 4666 images belonging to 5 classes.
Found 934 images belonging to 5 classes.


<ipython-input-39-a1173db000ed>:45: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history=model.fit_generator(train_generator,


Epoch 1/25
12/37 [========>.....................] - ETA: 38:10 - loss: 2.4307 - accuracy: 0.2598

In [ ]:
print("==============TEST RESULTS============")
test_generator = test_datagen.flow_from_directory(
        test_path,
        target_size=(img_rows, img_cols),
        batch_size=batch_size,

        class_mode=None,
        shuffle=False)
predictions = model.predict(test_generator, verbose=1)
yPredictions = np.argmax(predictions, axis=1)
true_classes = test_generator.classes

testAcc,testPrec, testFScore = my_metrics(true_classes, yPredictions)

==============TEST RESULTS============
Found 1164 images belonging to 5 classes.
10/10 [==============================] - 4s 395ms/step
Accuracy  : 0.6941580756013745
Precision : 0.694342261053289
f1Score : 0.6918780588654657
[[ 85  36  16  11   4]
 [ 72 264  29  35  19]
 [ 14   3 309   2   0]
 [ 15  42   1  44  19]
 [  1  23   1  13 106]]
